# 2장 시계열 데이터 다루기

#### 필요 패키지 로드

In [2]:
import pandas as pd
import io
import requests
from datetime import datetime

## 1. 발견된 시계열
### 데이터 불러오기
- YearJoined: 연간 회원상태
- emails: 회원별 주간 이메일 열람 누적 기록
- donation: 회원별 기부 순간 기록

In [40]:
YearJoined = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/year_joined.csv")
emails = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/emails.csv")
donations = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/donations.csv")

특정 회원에 대한 기록 n개 여부 판단

In [ ]:
YearJoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


이메일 열람 null 값 확인 및 특정 회원(998) 기록 확인

In [7]:
print(emails[emails.emailsOpened < 1])
print(emails[emails.user == 998])

Empty DataFrame
Columns: [emailsOpened, user, week]
Index: []
       emailsOpened   user                 week
25464           1.0  998.0  2017-12-04 00:00:00
25465           3.0  998.0  2017-12-11 00:00:00
25466           3.0  998.0  2017-12-18 00:00:00
25467           3.0  998.0  2018-01-01 00:00:00
25468           3.0  998.0  2018-01-08 00:00:00
25469           2.0  998.0  2018-01-15 00:00:00
25470           3.0  998.0  2018-01-22 00:00:00
25471           2.0  998.0  2018-01-29 00:00:00
25472           3.0  998.0  2018-02-05 00:00:00
25473           3.0  998.0  2018-02-12 00:00:00
25474           3.0  998.0  2018-02-19 00:00:00
25475           2.0  998.0  2018-02-26 00:00:00
25476           2.0  998.0  2018-03-05 00:00:00
25477           3.0  998.0  2018-03-12 00:00:00
25478           2.0  998.0  2018-03-19 00:00:00
25479           2.0  998.0  2018-03-26 00:00:00
25480           3.0  998.0  2018-04-02 00:00:00
25481           3.0  998.0  2018-04-09 00:00:00
25482           3.0  998.0

특정 회원 기준 이메일 열람 최초 시점과 마지막 시점 사이 주간 수 계산
- 문자열 값을 날짜로 변환 후 계산
- `strptime`: 문자열 -> 날짜
- `strftime`: 날짜 -> 문자열

In [8]:
date_time_str_max = max(emails[emails.user == 998].week)
date_time_str_min = min(emails[emails.user == 998].week)

date_time_obj_max = datetime.strptime(date_time_str_max, '%Y-%m-%d %H:%M:%S')
date_time_obj_min = datetime.strptime(date_time_str_min, '%Y-%m-%d %H:%M:%S')

(date_time_obj_max - date_time_obj_min).days/7

25.0

In [9]:
emails[emails.user == 998].shape

(24, 3)

누락된 시간대를 삽입하기 위해,모든 회원과 주에 대한 합집합 조합인 다중 색인(`MultiIndex`) 색성
- 합집합 색인을 기존 이메일 자료의 index로 설정하고 누락된 값은 0으로 채우기

In [ ]:
complete_idx = pd.MultiIndex.from_product([set(emails.week), set(emails.user)])

In [30]:
'''완전한 시계열 데이터 생성''' 
all_email = emails\
    .set_index(['week', 'user'])\
    .reindex(complete_idx, fill_value = 0)\
    .reset_index()
all_email.columns = ['week','user','emailsOpened']
all_email['week'] = pd.to_datetime(all_email['week'])
all_email[all_email.user == 998].sort_values('week')

,week,user,emailsOpened
22098,2015-02-09,998.0,0.0
70608,2015-02-16,998.0,0.0
80310,2015-02-23,998.0,0.0
66296,2015-03-02,998.0,0.0
61984,2015-03-09,998.0,0.0
...,...,...,...
54977,2018-04-30,998.0,3.0
16708,2018-05-07,998.0,3.0
28027,2018-05-14,998.0,3.0
5928,2018-05-21,998.0,3.0


회원별 최초 및 마지막 열람시기 확인
- 회원별 최초 열람시기 이전 타임스탬프 제거
- 회원별 최종 열람시기 이후 타임스탬프 제거

In [31]:
cutoff_dates = emails.groupby('user').week.agg(['min', 'max']).reset_index()

import warnings
warnings.filterwarnings('ignore')

for _, row in cutoff_dates.iterrows():
    user = row['user']
    start_date = row['min']
    end_date = row['max']
    # 최초 열람 이전 시기 제거
    all_email.drop(all_email[all_email.user == user][all_email.week < start_date].index, inplace=True)
    # 최종 열람 이후 시기 제거
    all_email.drop(all_email[all_email.user == user][all_email.week > end_date].index, inplace=True)

#### 즉, 위 과정들은 회원별 이메일 열람 데이터 중 최초-최종 타임스탬프 내 비어있는 시간대를 0으로 채워넣기 위한 과정이었다.

### 기부자료 주단위로 다운샘플링
1. 회원별 기부자료 날짜 데이터 형식 변환
2. 타임스탬프를 index로 설정
3. 주단위 월요일(`W-MON`) 로 재구성

`resample` 함수는 시계열 자료를 다운 및 업샘플링함
- 주단위(`W`), 월단위(`M`)
- 주단위 기본값은 일요일

In [41]:
donations.timestamp = pd.to_datetime(donations.timestamp)
donations.set_index('timestamp', inplace=True)
agg_don = donations.groupby('user').apply(lambda df: df.amount.resample("W-MON").sum().dropna())

### 동일한 시간빈도의 2자료 결합
- 회원별 이메일 열람기록과 기부자료

#### Multiindex 자료 내 특정 레이블 추출
- `get_level_values()` 활용
- `MultiIndex.get_leval_values('index_name')`
- 또는 조건 대상이 되는 index의 위치활용: `get_level_values(0)`

In [58]:
'''주간 이메일 및 기부자료 결합'''
merged_df = pd.DataFrame()

for user, user_email in all_email.groupby('user'):
    # 특정 회원 기부자료 추출 후 index 제거
    user_donations = agg_don[agg_don.index.get_level_values('user') == user].droplevel('user')
    
    user_email = all_email[all_email.user == user]\
        .sort_values('week')\
        .set_index('week')
        
    df = pd.merge(user_email, user_donations, how='left', left_index=True, right_index=True)
    df.fillna(0, inplace=True)

    merged_df = pd.concat([merged_df, df.reset_index()[['user', 'week', 'emailsOpened', 'amount']]])

#### 회원별 이메일 응답이 다음 기부에 미치는 영향 파악
- 특정 주의 기부는 기부가 발생하기 전 주인 이메일 열람 기록을 고려해야 한다.
- 따라서, 기부 정보를 한 시점 뒤로 미루기 위해 `shift(n)` 함수 활용
- `shift()`는 기본적으로 시간 빈도를 뒤로 미룸. 즉, 다음 시간대로 이동시킴

In [61]:
df = merged_df[merged_df.user == 998]
df['target'] = df.amount.shift(1)
df = df.fillna(0)

# 2. 데이터 정리

## 2.1 누락 데이터 처리
### 포워드필

### 이동평균

### 보간법

## 2.2 데이터 평활

pandas의 `ewm` 함수를 이용하여 데이터 평활 진행
- `ewm`이란 지수가중함수로 오래된 데이터에 지수감쇠를 적용하여 최근 데이터가 더 큰 영향을 끼지도록 가중치를 주는 함수
- 보통 추가 메서드로 `mean()` 활용하여 지수가중평균 적용
- `alpha` 매개변수를 이용하여 최신값 활용 가중치 조절
- 클수록 현재값에 빠르게 가까워짐

In [64]:
air = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/AirPassengers.csv", names=['Date', 'Passengers'])

In [65]:
air['Smooth.5'] = air.ewm(alpha = .5).Passengers.mean()
air['Smooth.9'] = air.ewm(alpha = .9).Passengers.mean()

# 3. 타임스탬프 이해
- datetime 모듈에는 기본적으로 시간대 정보가 없어서 지정이 필요하다.

In [72]:
# 64 page
import datetime
print(datetime.datetime.utcnow())
print(datetime.datetime.now())
print(datetime.datetime.now(datetime.timezone.utc))

2025-11-23 09:10:12.798811
2025-11-23 18:10:12.798869
2025-11-23 09:10:12.798922+00:00


#### 시간대 지정
- 미국 태평양 시간대(`US/Pacific`)라는 객체를 생성한 후, datetime 시간 객체를 `US/Pacific` 시간대 기준으로 현지화
- 다시 말해, datetime 모듈 내 시간정보는 시간대 정보가 없어 표준시간대별 다르게 이해하기 때문에 시간대 설정이 필요하다.
- 따라서, 해당 시간대를 미국 태평양 시간대 기준의 시간임을 지정한 것이다.

In [71]:
import pytz

western = pytz.timezone('US/Pacific')
western.zone

'US/Pacific'

In [73]:
loc_dt = western.localize(datetime.datetime(2018, 5, 15, 12, 34, 0))
loc_dt

datetime.datetime(2018, 5, 15, 12, 34, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

미국 태평양 시간대 기준이었던 시간을 런던 시간대로 변환

In [74]:
london_tz = pytz.timezone('Europe/London')
london_dt = loc_dt.astimezone(london_tz)
london_dt

datetime.datetime(2018, 5, 15, 20, 34, tzinfo=<DstTzInfo 'Europe/London' BST+1:00:00 DST>)

In [80]:
f = '%Y-%m-%d %H:%M:%S %Z%z'
print(datetime.datetime(2018, 5, 15, 12, 34, 0, tzinfo = london_tz).strftime(f))
print(london_tz.localize(datetime.datetime(2018, 5, 15, 12, 34, 0)).strftime(f))

2018-05-15 12:34:00 LMT-0001
2018-05-15 12:34:00 BST+0100
